In [1]:
import z3
from lib.ts import Ts
from lib.prove import prove_inductive

In [2]:
# The smart contract as a transition system.
ts = Ts('wallet')

owner = 100
sender = z3.BitVec('sender', 16)
receiver = z3.BitVec('receiver', 16)
amount = z3.Int('amount')
p = z3.BitVec('p',16)
address = z3.BitVec('address',16)
totalSupply, totalSupplyOut = ts.add_var(z3.IntSort(), name='totalSupply')

# Balances as an array, mapping from address to Integer.
balances, balancesOut = ts.add_var(z3.ArraySort(z3.BitVecSort(16), z3.IntSort()), name='balance')


In [3]:
# Transitions

ts.Init = z3.And(totalSupply == 0, z3.ForAll(p, balances[p]==0))

tr_mint = z3.And(sender==owner, amount > 0, totalSupplyOut == totalSupply + amount, 
                address > 0,
               balancesOut == z3.Update(balances,address,balances[address]+amount))

tr_burn = z3.And(sender==owner, amount > 0, amount <= balances[p], 
                totalSupplyOut == totalSupply - amount,
                balancesOut == z3.Update(balances,p,balances[p]-amount))


tr_transfer = z3.And(balances[sender] >= amount, amount > 0, 
                    balancesOut == z3.Update(z3.Update(balances,sender,balances[sender]-amount), 
                        receiver, balances[receiver]+amount))

ts.Tr = z3.Or(tr_mint, tr_burn, tr_transfer)


In [4]:
# Property as an inductive invariant. 
ts_property = z3.ForAll(p, balances[p] >= 0)

In [5]:
prove_inductive(ts,ts_property)

Prove init => property.
proved
Prove property is inductive.
proved
